In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import metrics,preprocessing
import seaborn as sns
import numpy as np
from sklearn import 
import matplotlib.pyplot as plt
from boruta import BorutaPy
from sklearn.naive_bayes import GaussianNB
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,RandomForestRegressor
import joblib
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_pickle('C:\\Users\\Nicol\\Documents\\Master\\SecondoProgetto\\data.2021-01-29.scenarioJunction.pkl.gz')

In [25]:
def dfColor(df, color):
    df_new = pd.concat([df[[c for c in df.columns if color in c]], df['winner']], axis=1)
    df_new.loc[df.winner == color, "winner"] = +1
    df_new.loc[df.winner != color, "winner"] = -1
    return df_new

def dfRegressor(df):
    df_red = dfColor(df, "red")
    df_blue = dfColor(df, "blue")
    random_regressor(df_red, "red")
    random_regressor(df_blue, "blue")
def dfCross(df):
    df_red = dfColor(df, "red")
    df_blue = dfColor(df, "blue")
    regression_cross(df_red, "red")
    regression_cross(df_blue, "blue")

In [27]:
def random_regressor(df,color):
    X = df.drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")
    y = df['winner']
    train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c = df.select_dtypes(include=['object']).drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'],
                                                  axis=1, errors="ignore").columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    regressors = [
        RandomForestRegressor(n_estimators=1000, random_state=42),
    ]

    for regressor in regressors:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                               ('scale',StandardScaler()),
                               ('regressor', regressor)])
        pipe.fit(train_features, train_labels)
        
        # Use the forest's predict method on the test data
        predictions = pipe.predict(test_features)
        # Calculate the absolute errors
        errors = abs(predictions - test_labels)
        # Print out the mean absolute error (mae)
        print('COLOR',color)
        print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
        # Calculate mean absolute percentage error (MAPE)
        mape = 100 * (errors / test_labels)
        # Calculate and display accuracy
        accuracy = 100 - np.mean(mape)
        #print('Accuracy:', round(accuracy, 2), '%.')
        print('Mean Absolute Error:', metrics.mean_absolute_error(test_labels, predictions))
        print('Mean Squared Error:', metrics.mean_squared_error(test_labels, predictions))
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_labels, predictions)))
        
        '''
        importance = pipe['regressor'].feature_importances_
        # summarize feature importance
        for i,v in enumerate(importance):
            print('Feature: %0d, Score: %.5f' % (i,v))
        # plot feature importance
        pyplot.bar([x for x in range(len(importance))], importance)
        pyplot.show()
        '''

In [59]:
def regression_cross(df,color):
    X = df.drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")
    y = df['winner']

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c = df.select_dtypes(include=['object']).drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'],
                                                  axis=1, errors="ignore").columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    regressor =  RandomForestRegressor(random_state = 10)
    
    param_grid = {
       'regressor__n_estimators': [100, 200, 500],
       'regressor__max_features': ['auto', 'sqrt', 'log2'],
       'regressor__max_depth' : [4,5,6,7,8],
       'regressor__criterion' :['mse', 'mae']
    }
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('scale',StandardScaler()),
                           ('regressor', regressor)])
        
   
    GSCV = GridSearchCV(pipe, param_grid=param_grid, cv=5)

    GSCV.fit(X, y)
    #GSCV.best_params_ 
    print("Best cross-validation score: {:.2f}".format(GSCV.best_score_))


    print('COLOR',color)
    
   

In [62]:
dfRegressor(df.sample(100000))

COLOR red
Mean Absolute Error: 0.8 degrees.
Mean Absolute Error: 0.8037995728960073
Mean Squared Error: 0.8014033119726484
Root Mean Squared Error: 0.8952113225225921
COLOR blue
Mean Absolute Error: 0.85 degrees.
Mean Absolute Error: 0.8489934285908953
Mean Squared Error: 0.8473732470789807
Root Mean Squared Error: 0.9205287866650237


In [60]:
dfCross(df.sample(10))

Best cross-validation score: -0.25
COLOR red
Best cross-validation score: -0.35
COLOR blue


In [ ]:
def random_forest(df):
    f_select=[]
   
    X = df.drop('winner',1)
    y = df['winner'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    proba = clf.predict_proba(X_test)
    proba = np.around(proba, decimals=2)
    print(y_train)


    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # feature importanti per determinarlo
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    print('Feature importance:\n', feature_imp)
    f_selection=feature_imp[(feature_imp>0)]
    print('Feature selection:\n',f_selection)
            
        
    # Creating a bar plot
    sns.barplot(x=f_selection, y=f_selection.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()
    #return X_test, y_test, y_pred, proba


# Label Encoder


In [ ]:
le_df=df
le = preprocessing.LabelEncoder()
le_df[c] = df[c].apply(lambda col: le.fit_transform(col))

In [ ]:
le_df

# One Hot Encoder

In [ ]:
ohe_df=df
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(ohe_df[c]).toarray())
# merge with main df bridge_df on key values
ohe_df = ohe_df.join(enc_df)
ohe_df

In [ ]:
lb = preprocessing.LabelBinarizer()
print ("Label Binarizer:", lb.fit_transform(df['team_red_2_0']))

In [ ]:
lb_df=df
lb = preprocessing.LabelBinarizer()
lb_df[c] = df[c].apply(lambda col: lb.fit_transform(col))


# Dummies

In [ ]:
bridge_df=df
X = pd.get_dummies(bridge_df, prefix_sep='_', drop_first=True)
bridge_df = bridge_df.merge(X)
bridge_df

In [ ]:
X

In [ ]:
plt.scatter(df.index,df['kind_red_Tank1'])

In [ ]:
random_forest(le_df)

In [ ]:
boruta(df)

linear_svm(df)

In [ ]:
linear_svm1(le_df)

In [ ]:
naive_bayes(le_df)

In [ ]:
linear_svm(df)